In [1]:
import os
import glob
import pandas as pd
import pickle
from scipy.stats import zscore
from collections import OrderedDict

Purpose: wrangle the physiology metrics that were output by mousephgymetrics.

Python environment: clustering

Note: run this script from the folder in which it is located so the relative paths work correctly

# Load data

In [2]:
#load the physiological metrics that we computed from the resp and pleth traces (e.g. HR, RR, PVI, HRV, RV, RRV)
final_data_folder='../phgy_derivatives/physiology_analysis_outputs'
resp_csvs = sorted(glob.glob(final_data_folder + '/*resp*window.csv'))
pulseox_csvs = sorted(glob.glob(final_data_folder + '/*pleth*window.csv'))

#for spo2, we didn't need to compute any downstream metrics, the original (cleaned) data is already the output that we need
spo2_csvs = sorted(glob.glob('../../3_preprocessed_data/phgy_data_oldNaming/*spo2*'))

#also load the motion measure (FD) computed by rabies
rabies_FD_csv_path = '/data/chamal/projects/mila/2021_fMRI_dev/part2_phgy_fmri_project/4_derivatives/rabies_runs/local_data_final_runs/rabies_out_preprocess-v050/motion_datasink/FD_csv/*/*/'

#load the timeseries of fmri censoring, so that we can also censor the phgy timeseries to get the same number of timepoints for fmri and phgy
rabies_censoring_csv_path = '/data/chamal/projects/mila/2021_fMRI_dev/part2_phgy_fmri_project/4_derivatives/rabies_runs/local_data_final_runs/rabies_out_cc-v050_05smoothed_lowpass/confound_correction_datasink/frame_censoring_mask/*/' 

#finally, load the dataset specs
dataset_info_df = pd.read_csv("../../2_raw_data/dataset_specs.csv").set_index("epi_filename")

# Define function

In [3]:
def extract_array_of_phgy_with_dict(resp_files, pulseox_files, spo2_files, rabies_censoring_csv_path,
                                    rabies_FD_csv_path,range_files_to_extract,
                                    output_dict_bool, original_num_timepoints, censoring_duration_dict, dataset_info_df):
    '''This function combines the resp and pulseox csvs for a given subject/session, drops unimportant variables
    and censors according to the RABIES temporal mask. Will also do additional physiology censoring (remove 
    corrupted timepoints) and drop NaN values. All while properly keeping track of the shape. '''
    phgy_dict_allvar = {}
    phgy_dict_selectvar = {}
    phgy_dict_selectvar_precensor = {}
    phgy_dict_selectvar_zscored = {}
    phgy_nan_censoring_dict = {}
    fmri_phgy_nan_censoring_dict = {}
    phgy_shape_postCensor_dict = {}
    session_info_dict = {}
    
    
    #iterate over all csvs and extract an array from each, store outputs in a dict
    for i in range(range_files_to_extract[0], range_files_to_extract[1]):
        num_files_to_extract = range_files_to_extract[1] - range_files_to_extract[0]
        filename = os.path.basename(os.path.abspath(resp_files[i]))
        ses = filename[2:3]
        sub = filename[4:7]
        sub_ses_ID = 'sub-PHG' + str(sub) + '_ses-' + str(ses)
        
        #find corresponding rabies csv
        rabies_censoring_csv = sorted(glob.glob(str(rabies_censoring_csv_path) + '/' + (sub_ses_ID) + '*.csv'))
        rabies_FD_csv = sorted(glob.glob(str(rabies_FD_csv_path) + '/' + (sub_ses_ID) + '*rest*.csv')) 
        
        #load as df
        resp_df = pd.read_csv(resp_files[i]).drop(columns = ['Unnamed: 0', 'Window start time',
                                                            'Window end time'], axis = 1)
        pulseox_df = pd.read_csv(pulseox_files[i]).drop(columns = ['Unnamed: 0', 'Window start time',
                                                            'Window end time'], axis = 1)
        spo2_df = pd.read_csv(spo2_files[i], names = ['SpO2 (%)'])
        rabies_censoring_df = pd.read_csv(os.path.abspath(rabies_censoring_csv[0]))
        rabies_FD_df = pd.read_csv(os.path.abspath(rabies_FD_csv[0]))

        #concatenate resp, pulseox, spo2 and FD together
        phgy_df_allvar = pd.concat([resp_df.add_suffix('-resp'), pulseox_df.add_suffix('-pleth'), spo2_df], axis = 1)
        
        #extract relevant information about the scan to keep, repeat it for each timepoint
        session_info_once = pd.DataFrame(dataset_info_df.loc[sub_ses_ID][0:13]).transpose()
        session_info_repeated = session_info_once.loc[session_info_once.index.repeat(original_num_timepoints)].reset_index()
        
        #extract the iso info, repeat it appropriately for each iso condition at each timepoint
        iso_info = pd.DataFrame([dataset_info_df.loc[sub_ses_ID,:]['iso_percent_low']]*480 + [dataset_info_df.loc[sub_ses_ID,:]['iso_percent_mid']]*480 + [dataset_info_df.loc[sub_ses_ID,:]['iso_percent_high']]*480, 
                                columns = ['isoflurane_percent'])   
        
        #also keep track of the time (how far into the scan we are)
        time1 = pd.DataFrame([*range(0,1440)], columns = ['Time after scan start'])
        time2 = pd.DataFrame([*range(0,480)]*3, columns = ['Time after isoflurane change'])
        
        #concatenate the session_info and iso_info and FD
        full_session_info = pd.concat([time1, time2, session_info_repeated, iso_info, rabies_FD_df.add_suffix(' FD')],
                                      axis = 1)
        
        
        #drop redundant variables that are highly correlated (see next section for the justification)
        #also drop std in window, entropy and pulse width because they're harder to interpret and less grounded in lit
        phgy_df_selectvar = phgy_df_allvar.drop(columns = ['Period-overall window mean-resp',
                                                           'Resp rate-overall window-resp',
                                                           'Instantanous RRV period rmssd-window mean-resp',
                                                           'RRV-overall period window rmssd-resp',
                                                           'RRV-overall period window std-resp',
                                                           'Period-overall window mean-pleth',
                                                           'HR-overall window-pleth',
                                                           'Instantanous HRV period rmssd-window mean-pleth',
                                                           'HRV-overall period window rmssd-pleth',
                                                           'HRV-overall period window std-pleth',
                                                           'Instantaneous resp rate - window std-resp',
                                                          'Instantaneous HR - window std-pleth',
                                                          'Instantaneous entropy-window mean-resp',
                                                          'Instantaneous entropy-window mean-pleth',
                                                          'Width at quarter height-overall window mean-pleth',
                                                          'width at half height-overall window mean-pleth',
                                                          'Width at base-overall window mean-pleth'], axis = 1)
        #rename the existing column names to simpler ones
        phgy_df_selectvar = phgy_df_selectvar.rename(columns={"Instantaneous resp rate-window mean-resp": "RR",
                                                              "Instantaneous RV - window mean-resp": "RV", 
                                                              "Instantaneous RRV period std-window mean-resp": "RRV",
                                                              "Instantaneous HR-window mean-pleth": "HR",
                                                              "Instantaneous HRV period std-window mean-pleth": "HRV",
                                                              "Instantaneous PVI-window mean-pleth": "PVI", 
                                                             "SpO2 (%)": "SpO2"})
        #change order of RV and RRV (to better parallel the order of pulse ox metrics)
        phgy_df_selectvar.insert(1, 'RRV', phgy_df_selectvar.pop('RRV'))
        phgy_df_selectvar = phgy_df_selectvar.astype(float)
        
        ################################ CENSORING ##################################
        #drop rows in phgy data that were censored by rabies
        phgy_df_allvar_fmriCensored = phgy_df_allvar.loc[rabies_censoring_df['False = Masked Frames'], :].reset_index(drop = True)
        phgy_df_selectvar_fmriCensored = phgy_df_selectvar.loc[rabies_censoring_df['False = Masked Frames'], :].reset_index(drop = True)
        
        #drop rows in full_session_info that were censored by rabies
        full_session_info_fmriCensored = full_session_info.loc[rabies_censoring_df['False = Masked Frames'], :].reset_index(drop = True)
        
        #create a boolean list of 'True' for each timepoint of the scan (this is the same format as rabies csvs)
        phgy_censoring_list = [True] * original_num_timepoints
        #Then, for certain sessions, censor the session (or part of it) by changing the True to False
        #first, check if this particular subject is in the list of ones that need to be censored
        list_censoring_duration_per_sub = list(censoring_duration_dict.keys())
        if sub_ses_ID in list_censoring_duration_per_sub:
            #if so, extract the censor start and end time
            censor_starttime = censoring_duration_dict[sub_ses_ID][0]
            censor_endtime = censoring_duration_dict[sub_ses_ID][1]
            #modify the phgy censoring list for this subject so that those timepoints now read false
            phgy_censoring_list[censor_starttime : censor_endtime] = [False] * (censor_endtime-censor_starttime)
        phgy_censoring_df = pd.DataFrame(phgy_censoring_list)
        
        #there are still some remaining NaN rows, I want to mask out any row with these because it won't run the clustering
        nan_censoring_df_tmp = phgy_df_selectvar.isna().any(axis = 1).reset_index(drop=True) 
        nan_censoring_df = (nan_censoring_df_tmp == False)
        
        #combine nan_censoring_df and phgy_censoring_df into one censoring_df
        phgy_nan_censoring_df = (nan_censoring_df) & (phgy_censoring_df[0])
        fmri_phgy_nan_censoring_df = rabies_censoring_df['False = Masked Frames'] & (nan_censoring_df) & (phgy_censoring_df[0])
        
        #now, because we were working with the original number of timepoints (because we determined censor times based on the phgyQC images), also drop those that were removed during fmri-based censoring
        phgy_nan_censoring_df_final = phgy_nan_censoring_df.loc[rabies_censoring_df['False = Masked Frames']].reset_index(drop = True)

        #finally, apply the phgy and nan censoring to the phgy data and session_info
        phgy_df_selectvar_allCensored = phgy_df_selectvar_fmriCensored.loc[phgy_nan_censoring_df_final, :]
        full_session_info_allCensored = full_session_info_fmriCensored.loc[phgy_nan_censoring_df_final, :]
        
        #################################### ZSCORE ON A PER SUBJECT BASIS ###################################
        #the apply function will only work if the dataframe is not completely censored (ie not empty)
        if phgy_df_selectvar_allCensored.shape[0] != 0:
            phgy_df_selectvar_allCensored_zscored = phgy_df_selectvar_allCensored.apply(zscore)
        else:
            #if it is empty, no need to zscore
            phgy_df_selectvar_allCensored_zscored = phgy_df_selectvar_allCensored
        
        #save in dict
        phgy_dict_allvar[sub_ses_ID] = phgy_df_allvar_fmriCensored
        phgy_dict_selectvar[sub_ses_ID] = phgy_df_selectvar_allCensored
        phgy_dict_selectvar_precensor[sub_ses_ID] = phgy_df_selectvar
        phgy_dict_selectvar_zscored[sub_ses_ID] = phgy_df_selectvar_allCensored_zscored
        phgy_shape_postCensor_dict[sub_ses_ID] = phgy_df_selectvar_allCensored.shape[0]
        phgy_nan_censoring_dict[sub_ses_ID] = phgy_nan_censoring_df_final
        fmri_phgy_nan_censoring_dict[sub_ses_ID] = fmri_phgy_nan_censoring_df
        session_info_dict[sub_ses_ID] = full_session_info_allCensored
        
        #sort the dict so it has same order as fmri dict
        phgy_dict_selectvar_sorted = OrderedDict(sorted(phgy_dict_selectvar.items()))
        phgy_dict_selectvar_precensor_sorted = OrderedDict(sorted(phgy_dict_selectvar_precensor.items()))
        phgy_dict_selectvar_zscored_sorted = OrderedDict(sorted(phgy_dict_selectvar_zscored.items()))
        phgy_shape_postCensor_dict_sorted = OrderedDict(sorted(phgy_shape_postCensor_dict.items()))
        phgy_nan_censoring_dict_sorted = OrderedDict(sorted(phgy_nan_censoring_dict.items()))
        fmri_phgy_nan_censoring_dict_sorted = OrderedDict(sorted(fmri_phgy_nan_censoring_dict.items()))
        session_info_dict_sorted = OrderedDict(sorted(session_info_dict.items()))
        
        #print the progress
        print("\r", 'loading file ' + str(i+1)  + ' out of ' + str(num_files_to_extract), end="")
    
    #concatenate each file stored as an element of the dict
    if output_dict_bool:
        return phgy_dict_selectvar_precensor_sorted, phgy_dict_allvar, phgy_dict_selectvar_sorted, phgy_dict_selectvar_zscored_sorted, phgy_shape_postCensor_dict_sorted, phgy_nan_censoring_dict_sorted, fmri_phgy_nan_censoring_dict_sorted, session_info_dict_sorted
    else:
        phgy_arr_selectvar_zscored = pd.concat([phgy_dict_selectvar_zscored_sorted[x] for x in phgy_dict_selectvar_zscored_sorted], 0)
        phgy_arr_selectvar_precensor = pd.concat([phgy_dict_selectvar_precensor_sorted[x] for x in phgy_dict_selectvar_precensor_sorted], 0)
        phgy_arr_selectvar = pd.concat([phgy_dict_selectvar_sorted[x] for x in phgy_dict_selectvar_sorted], 0)
        phgy_arr_allvar = pd.concat([phgy_dict_allvar[x] for x in phgy_dict_allvar], 0)
        phgy_nan_censoring_arr = pd.concat([phgy_nan_censoring_dict_sorted[x] for x in phgy_nan_censoring_dict_sorted], 0)
        fmri_phgy_nan_censoring_arr = pd.concat([fmri_phgy_nan_censoring_dict_sorted[x] for x in fmri_phgy_nan_censoring_dict_sorted], 0)
        session_info_arr = pd.concat([session_info_dict_sorted[x] for x in session_info_dict_sorted], 0)
        
        return phgy_arr_selectvar_precensor, phgy_arr_allvar, phgy_arr_selectvar, phgy_arr_selectvar_zscored, phgy_shape_postCensor_dict_sorted, phgy_nan_censoring_arr, fmri_phgy_nan_censoring_arr, session_info_arr


# Censor the physiology data and combine into one csv

Censor according to a) timepoints that don't pass the phgy QC, b) timepoints that are censored in the fmri timeseries
due to motion, c) drop nan values that could not be estimated during phgy metric computation. Combine remaining phgy metrics into 1 csv per subject and session.

In [4]:
#manually selected timepoints to censor from the physiology data - based on visual QC
censor_duration_dict = {'sub-PHG004_ses-1': [0,1440], 'sub-PHG001_ses-2': [0,1440], 'sub-PHG005_ses-2': [0,1440],
                           'sub-PHG004_ses-2': [1120,1440], 'sub-PHG013_ses-2': [1380,1440],
                          'sub-PHG003_ses-3': [475,1440], 'sub-PHG007_ses-1': [0,1440], 
                           'sub-PHG008_ses-1': [0,450], 'sub-PHG016_ses-3': [1380,1440],
                       'sub-PHG010_ses-3': [0,1440]}
#sub-PHG010_ses-3 is also not there because the spo2 (rates) data is not available

In [5]:
# perform censoring, save outputs into arrays
phgy_arr_selectvar_precensor, phgy_arr_allvar, phgy_arr_selectvar, phgy_arr_selectvar_sub_zscored, phgy_dict_postCensor_shape, phgy_nan_censor, fmri_phgy_nan_censor, session_info_arr = extract_array_of_phgy_with_dict(resp_csvs, 
                                                                                                   pulseox_csvs, spo2_csvs, rabies_censoring_csv_path, rabies_FD_csv_path,
                                                                                                   [0,46], False,1440,
                                                                                                   censor_duration_dict, dataset_info_df)


 loading file 46 out of 46

In [ ]:
#save large files as pickle files so they can be easily loaded later without rerunning everything
pickle.dump(phgy_dict_postCensor_shape, open("./pickle_files/phgy_dict_postCensor_shape_withEdgeCutoff", "wb"))
pickle.dump(phgy_nan_censor, open("./pickle_files/bool_series_phgy-nan_censoring_withEdgeCutoff", "wb"))
pickle.dump(fmri_phgy_nan_censor, open("./pickle_files/series_fmri_phgy_nan_censor", "wb"))

In [7]:
#run censoring, save outputs as dict
phgy_dict_selectvar_precensor, phgy_dict_allvar, phgy_dict_selectvar, phgy_dict_selectvar_sub_zscored, phgy_dict_postCensor_shape, phgy_nan_censor, fmri_phgy_nan_censor_dict, session_info_dict = extract_array_of_phgy_with_dict(resp_csvs, 
                                                                                                   pulseox_csvs, spo2_csvs, rabies_censoring_csv_path,rabies_FD_csv_path,
                                                                                                   [0,46], True,1440,
                                                                                                   censor_duration_dict, dataset_info_df)


 loading file 46 out of 46

In [ ]:
#save outputs as pickle files
pickle.dump(phgy_dict_selectvar, open("./pickle_files/phgy_dict_selectvar", "wb"))
pickle.dump(phgy_dict_selectvar_precensor, open("./pickle_files/phgy_dict_selectvar_precensor", "wb"))
pickle.dump(fmri_phgy_nan_censor_dict, open("./pickle_files/dict_fmri_phgy_nan_censor", "wb"))
pickle.dump(phgy_nan_censor, open("./pickle_files/dict_phgy_nan_censor", "wb"))

In [9]:
#check dimensions of outputs
print('Number of timepoints after removing timepoints also censored during fmri (should match total fmri timepoints), and number of phgy variables: ' + str(phgy_arr_allvar.shape))
print('Number of timepoints after further removing timepoints that dont pass phgy QC and nan values, and final number of selected phgy variables: ' + str(phgy_arr_selectvar.shape))

Number of timepoints after removing timepoints also censored during fmri (should match total fmri timepoints), and number of phgy variables: (61561, 24)
Number of timepoints after further removing timepoints that dont pass phgy QC and nan values, and final number of selected phgy variables: (52606, 7)


The numbers above checkout. 61561 is also the number of fmri timepoints (was 64055 before adding edge cutoff during fmri processing). 52606 is less than that (was 55200 before added edge cutoff in the fmri processing).

In [10]:
#save the final csv
phgy_arr_selectvar.to_csv("./phgy_metrics_final_csv/phgy_metrics_noZscore_selectvar_phgyfmriCensored.csv")
session_info_arr.to_csv("./phgy_metrics_final_csv/session_info_phgyfmriCensored.csv")